# Instant Gratification

## Your first Frontier LLM Project!

Let's build a useful LLM solution - in a matter of minutes.

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!




In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.


In [2]:
# # Load environment variables in a file called .env

# load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')

# # Check the key

# if not api_key:
#     print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
# # elif not api_key.startswith("sk-proj-"):
# #     print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
# elif api_key.strip() != api_key:
#     print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
# else:
#     print("API key found and looks good so far!")


In [3]:
MODEL = "llama3.2"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

# Let's make a quick call to the LLM model to get started, as a preview!

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, Llama! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(
 model=MODEL,
 messages=[{"role": "user", "content":message}])
print(response.choices[0].message.content)

Hi there! It's great to meet you! I'm excited to be your conversational AI companion. Don't worry if this is the start of our conversation - I'll do my best to make it a helpful and friendly one. How's your day going so far? Is there something specific you'd like to talk about or ask, or would you like me to suggest some topics to get us started?


## OK onwards with our first project

In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://www.investorgain.com/")
print(ed.title)
print(ed.text)

Best Online Share Brokers in India-2025
Open Account (FREE)
Zerodha (Trading Account)
FREE Equity Delivery and MF
Flat â‚¹20/trade Intra-day/F&O
Open Instant Account for FREE
Free Delivery Trading + Rs 20 for FnO
Ad
www.AngelOne.in
FREE Account Opening + Zero AMC Fees* + MutualFund SIP
GMP
IPO
SHARE BROKERS REVIEW
BEST BROKERS
BROKERAGE CALCULATOR
COMPARE BROKERS
×
Discount Brokers
Zerodha
(₹20/trade)
ProStocks
(₹899 Per
							Month)
5paisa
(₹20/trade)
Upstox
(₹20/trade)
Angel One
(₹20/trade)
More Discount Brokers
Offers & Compare
Top 10 Discount Brokers in India
Top 10 Stock Brokers in India
Top 5 commodity discount brokers in India
Special Offers
Side by Side Compare
FAQs
FAQ Home
Stock Brokers
IPOs
NRI
						Trading
Demat
Sovereign Gold Bond
×
Mainboard IPOs
IPO Home
Live IPO
						GMP
IPO Live Subscription
IPO
						Reports
SME IPOs
SME IPO
						Home
SME IPO Live Subscription
NCDs
NCDs
						Home
Right Issue
Right Issue Home
Buyback
Buyback
						Home
Sovereign Gold Bond (SGB)
SG

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

LLM models have been trained to receive instructions in a particular way. It can be found on model card in hugging face website

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(user_prompt_for(ed))

You are looking at a website titled Best Online Share Brokers in India-2025
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Open Account (FREE)
Zerodha (Trading Account)
FREE Equity Delivery and MF
Flat â‚¹20/trade Intra-day/F&O
Open Instant Account for FREE
Free Delivery Trading + Rs 20 for FnO
Ad
www.AngelOne.in
FREE Account Opening + Zero AMC Fees* + MutualFund SIP
GMP
IPO
SHARE BROKERS REVIEW
BEST BROKERS
BROKERAGE CALCULATOR
COMPARE BROKERS
×
Discount Brokers
Zerodha
(₹20/trade)
ProStocks
(₹899 Per
							Month)
5paisa
(₹20/trade)
Upstox
(₹20/trade)
Angel One
(₹20/trade)
More Discount Brokers
Offers & Compare
Top 10 Discount Brokers in India
Top 10 Stock Brokers in India
Top 5 commodity discount brokers in India
Special Offers
Side by Side Compare
FAQs
FAQ Home
Stock Brokers
IPOs
NRI
						Trading
Demat
Sovereign Gold Bond
×
Mainboard IPOs
IPO Home
Live IPO
				

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the might GPT (yet!)

In [10]:

messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]


In [11]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(
 model=MODEL,
 messages=messages
)
print(response.choices[0].message.content)

*sigh* Oh, wow. I'm so shocked you didn't already know this one. Fine. The answer to 2 + 2 is... *dramatic pause* ...4. Congratulations, you managed to not break a sweat with that one. Next question, please.


## And now let's build useful messages for LLM Model, using a function

In [12]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled Best Online Share Brokers in India-2025\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nOpen Account (FREE)\nZerodha (Trading Account)\nFREE Equity Delivery and MF\nFlat â‚¹20/trade Intra-day/F&O\nOpen Instant Account for FREE\nFree Delivery Trading + Rs 20 for FnO\nAd\nwww.AngelOne.in\nFREE Account Opening + Zero AMC Fees* + MutualFund SIP\nGMP\nIPO\nSHARE BROKERS REVIEW\nBEST BROKERS\nBROKERAGE CALCULATOR\nCOMPARE BROKERS\n×\nDiscount Brokers\nZerodha\n(₹20/trade)\nProStocks\n(₹899 Per\n\t\t\t\t\t\t\tMonth)\n5paisa\n(₹20/trade)\nUpstox\n(₹20/trade)\nAngel One\n(₹20/trade)\nMore Discount Brokers\nOffers

## Time to bring it together - the API for OpenAI is very simple!

In [14]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("https://www.investorgain.com/")

"Welcome to InvestorGain.com, a comprehensive online platform providing detailed reviews, information, and resources for individual investors, traders, and financial enthusiasts.\n\n**Broker Comparison**\n\nWe offer a unique broker comparison feature that allows you to compare and contrast various stock brokers in India based on their service offerings, trading platforms, brokerage charges, customer support, and more. Our exhaustive list of top-rated stock brokers includes well-established players like Zerodha, Upstox, Angel One, Kotak Securities, Paytm Money, 5Paisa, Fyers, and AliceBlue.\n\n**Broker Reviews**\n\nHere are some detailed reviews for popular stock brokers in India:\n\n1. **Zerodha**: Known for its low brokerage rates and user-friendly trading platform. Zerodha is a favorite among Indian retail investors.\n2. **Upstox**: Offers competitive brokerage rates with its flat-fee model, making it an attractive option for traders.\n3. **Angel One**: Provides free equity delivery 

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://www.investorgain.com/")

This is a comprehensive review website for stock brokers and investment platforms in India. Here's a breakdown of the key features:

**Features:**

1. **Broker Comparison**: The website allows comparison of various stock brokers, including Zerodha, Angel One, Upstox, Kotak Securities, Paytm Money, 5Paisa, Fyers, and Alice Blue.
2. **Broker Reviews**: Each broker's review is comprehensive, covering aspects like account opening fees, demat fees, trading fees, research reports, mobile app, customer support, and more.
3. **IPO Information**: The website provides up-to-date information on IPOs (Initial Public Offering), including details on the company, market lot, price range, allotment status, and investment options.
4. **IPO Subscription Status**: Users can check the subscription status of IPOs online.
5. **Trading Tips and Guides**: The website offers trading tips and guides for various markets, including stocks, ETFs, Mutual Funds, and commodity markets.
6. **Trader Communities**: The website has dedicated communities for traders to discuss market trends, share experiences, and get support from fellow traders.

**Key Benefits:**

1. **Free Resources**: Many of the brokerages offer free resources, such as research reports, training sessions, and mobile apps.
2. **Accurate Data**: The website promises accurate and up-to-date information on various financial markets.
3. **Independent Reviews**: The reviews are based on independent research, ensuring unbiased information for traders.

**Target Audience:**

1. **Beginner Investors**: This platform is ideal for novice investors who want to start their investment journey with an idea of the best stock brokers.
2. **Experienced Traders**: Seasoned traders can also benefit from this platform, as it provides insights into different brokerages and helps them make informed decisions.

**Reviews:**

The website has collected a vast number of reviews from satisfied customers, which are shared on its platform without any bias or promotion. These reviews highlight the pros and cons of each brokerage, helping users make informed decisions.

Overall, InvestorGain.com is an excellent resource for individuals looking to navigate the Indian financial market efficiently. Whether you're new to investing or a seasoned trader, this platform provides valuable insights into various stock brokages and investment options.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. Selenium implementation that gets around this. You'll need to read up on it (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors

But many websites will work just fine!

In [18]:
display_summary("https://cnn.com")

Here are the top stories from CNN:

1. **Biden's Last-Minute Prisoner Exchange Deal with Taliban Delayed**: The delay on a prisoner exchange deal between the US and Afghanistan, negotiated by President Biden last-minute, would be part of a larger agreement to stop a military drawdown.
2. **Trump Revokes Security Clearances of Former Officials Who Signed Hunter Biden Laptop Letter**: The move could create tension between Trump's administration and former officials who signed letters supporting or criticizing his 2020 election loss claims.
3. **Lawyers Advise LGBTQ+ Community to Take Precautions Under Trump's Executive Order**: A new rule allowing federal contractors to discriminate based on their sex, pregnancy, sexual orientation, gender identity, or national origin may lead to widespread harm for LGBTQ+ individuals in the US.
4. **TikTok Had an Incredible Wild Weekend: Users Say It Was 'a Wild and Bizarre Time'**: The popular social media app had a dramatic shift with new features, including a virtual "Dance Dance Room" game that was being tested on a limited scale in some groups.
5. **Malaysia's Mystifying Wild Cats Have Been Spotted for Years by Researchers** This article appears to have been automatically generated and does not contain specific details about Malaysia or its feline population.

Other top stories from CNN include:

* "Carbonated Water and Weight Loss: What You Need to Know"
* "Climate Change" 
* "The Panama Canal Changed the World"
* "In this Ski World Cup Event, the Winners Get Millions In Prize Winnings"

These are just a few of the many stories currently featured on the CNN website. To get the latest news, follow CNN's website or use their mobile app to stay up-to-date with the latest developments in world news.

In [19]:
display_summary("https://anthropic.com")

**Anthropic Website Summary**
=====================================

### Mission and Overview
Anthropic aims to create reliable, beneficial AI systems. The company was founded by an interdisciplinary team with experience in ML, physics, policy, and product.

### AI Research and Products
Anthropic research focuses on AI safety, including alignment, constitutional AI, and feedback mechanisms. The company releases AI models like Claude 3.5 Sonnet, which can be used for various applications.

### Recent Announcements

*   **Introducing new Claude 3.5 models**: "computer use" provides a more human-like experience.
*   **Claude 3.5 updates**: improved performance and capabilities in different areas.

### Products
Anthropic offers AI solutions like Claude, which can be integrated with other applications to create custom experiences.

### Company Information
*   **Location**: San Francisco
*   **Mission**: Generate research and create beneficial AI systems

**Note:** Some navigation links and sections related to customer information or general inquiries have been omitted.